In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import plotly.express as px

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalCrud import AnimalCrud   # ADDED: import your CRUD class

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name
username = "aacuser"
password = "Securepasslol"

# Connect to database via CRUD Module
db = AnimalCrud(username, password)   # CHANGED: instantiate AnimalCrud instead of AnimalShelter

# ADDED: helper to load any query, swapping in a non-empty default if needed
def load_data(query: dict):
    if not query:
        query = {"_id": {"$exists": True}}
    records = db.read(query)
    df_tmp = pd.DataFrame.from_records(records)
    if "_id" in df_tmp.columns:
        df_tmp.drop(columns=['_id'], inplace=True)
    return df_tmp

# ADDED: mapping from our radio‐button values into MongoDB filters
def make_query(filter_type):
    specs = {
        "water": (
            ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"],
            "Intact Female", 26, 156
        ),
        "mountain": (
            ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
             "Siberian Husky", "Rottweiler"],
            "Intact Male", 26, 156
        ),
        "disaster": (
            ["Doberman Pinscher", "German Shepherd", "Golden Retriever",
             "Bloodhound", "Rottweiler"],
            "Intact Male", 20, 300
        )
    }
    if filter_type in specs:
        breeds, sex, min_wk, max_wk = specs[filter_type]
        return {
            "animal_type": "Dog",
            "breed": {"$in": breeds},
            "sex_upon_outcome": sex,
            "age_upon_outcome_in_weeks": {"$gte": min_wk, "$lte": max_wk}
        }
    return {}   # ADDED: empty dict causes load_data to fetch all records

# sending the read method an empty document requests all documents be returned
df = load_data({})

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo


app.layout = html.Div([
    html.A(
        html.Img(src="https://learn.snhu.edu/content/enforced/1918330-CS-340-10797.202551-1/course_documents/Grazioso%20Salvare%20Logo.png?_&d2lSessionVal=ScQoRYLN9OTGv4T9RbCyYwie6&ou=1332057&ou=1918330", height="60px"),
        href="https://www.snhu.edu",
        target="_blank",      # optional: open in new tab
        style={"display": "inline-block"}
    ),
     html.P(
        "Created by Brendan Clarke",
        style={'textAlign': 'center', 'fontStyle': 'italic'}
    ),
    

    #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue',               'value': 'water'},
            {'label': 'Mountain/Wilderness Rescue','value': 'mountain'},
            {'label': 'Disaster/Tracking Rescue',  'value': 'disaster'},
            {'label': 'Reset (show all)',          'value': 'reset'}
        ],
        value='reset',
        inline=True
    ),  # ADDED: rescue-type radio buttons

    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,            # ADDED: pagination
        sort_action="native",    # ADDED: client-side sorting
        filter_action="native",  # ADDED: client-side filtering
        style_table={"overflowX": "auto"}
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
    ),
    html.Br(),
    html.Hr(),

    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            html.Div(id='graph-id', className='col s12 m6'),
            html.Div(id='map-id',   className='col s12 m6')
        ]
    )
])

#############################################
# Interaction Between Components / Controller
#############################################

# ADDED: callback to refilter table when user picks a rescue-type
@app.callback(
    Output('datatable-id', 'data'),
    Input('filter-type', 'value')
)
def update_dashboard(filter_type):
    query = make_query(filter_type)
    df_filtered = load_data(query)
    return df_filtered.to_dict('records')

# Display the breeds of animal based on quantity represented in the data table
@app.callback(
    Output('graph-id', 'children'),
    Input('datatable-id', 'data')
)
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    if not viewData:
        return html.P("No data to display")
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
            figure=px.pie(
                dff,
                names='breed',
                title='Preferred Animals'
            )
        )
    ]  # ADDED: dynamic pie chart based on filtered data

# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    Input('datatable-id', 'selected_columns')
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', 'children'),
    Input('datatable-id', 'derived_virtual_data'),
    Input('datatable-id', 'derived_virtual_selected_rows')
)
def update_map(viewData, index):
    if viewData is None or index is None:
        return
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0
    # Austin TX is at [30.75, -97.48]
    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[30.75, -97.48],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                    children=[
                        dl.Tooltip(dff.iloc[row, 4]),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(dff.iloc[row, 9])
                        ])
                    ]
                )
            ]
        )
    ]

app.run_server(debug=False)


 * Running on http://127.0.0.1:27801/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Jun/2025 23:54:48] "GET /_alive_f887259e-05d4-467b-b46c-f9f7d5d6f731 HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:27801/
